In [1]:
using Random, JuMP, Ipopt, LinearAlgebra, SparseArrays, OSQP

const ETC_L1_QP_lib = "./ETC_L1_QP.so"
function call_ETC_L1_QP(L_Q_inv::Array{Float64,2},V::Array{Float64,2}, c::Array{Float64,1}, G::Array{Float64,2}, g::Array{Float64,1}, rho::Array{Float64,1}, epsilon::Float64, m::Int, n::Int)
    y = Array{Float64,1}(undef,m)
    run_time = Array{Float64,}(undef,1)
    ccall(
        (:ETC_L1_QP, ETC_L1_QP_lib),
        Cvoid,
        (Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}, Cdouble, Cint, Cint, Ptr{Cdouble}, Ptr{Cdouble}),
        L_Q_inv, V, c, G, g, rho, epsilon, m, n, y, run_time
    )
    return y, run_time    
end

call_ETC_L1_QP (generic function with 1 method)

In [2]:
Random.seed!(20240809)

num_condition = 20
num_randomQPs = 50

m = 20
n = 8*m

Worst_case_rel_err_Ipopt_L1_QP = []
Worst_case_sol_time_Ipopt_L1_QP = []

Worst_case_rel_err_ETC_L1_QP = []
Worst_case_sol_time_ETC_L1_QP = []

for each_condition in 1:num_condition
    cond_num = 1e1 * (1e6/1e1)^((each_condition-1)/(num_condition-1))
    rho = 1 * cond_num .* ones(n);

    Rel_err_Ipopt_L1_QP = []
    Sol_time_Ipopt_L1_QP = []

    Rel_err_ETC_L1_QP = []
    Sol_time_ETC_L1_QP = []

    for each_QP in 1:num_randomQPs
        U = rand(m,m)
        Q = U*Diagonal(rand(1:cond_num,m))*U'
        Q = 0.5*(Q+Q')
        c = cond_num*vec(rand(m,1))
        G = sprandn(n,m,0.8)
        G = Matrix(G)
        g = rand(n)

        # the base solution is calculated by Ipopt in solving the Original-QP
        model_0 = Model(Ipopt.Optimizer)
        set_attribute(model_0, "print_level", 0)
        set_attribute(model_0, "tol", 1e-12)
        @variable(model_0, y[1:m])
        @objective(model_0, Min, 0.5 * y' * Q * y + c'*y)
        for i in 1:n
            @constraint(model_0, dot(G[i,:], y) <= g[i])
        end
        optimize!(model_0)
        y = [value(y[i]) for i in 1:m];

        # Using Ipopt to solve the L1-QP
        H = G*inv(Q)*G'
        h = G*inv(Q)*c + g
        model = Model(Ipopt.Optimizer)
        set_attribute(model, "print_level", 0)
        set_attribute(model, "tol", 1e-12)
        @variable(model, 0.0 <= z[i=1:n]<= rho[i])
        @objective(model, Min, 0.5 * z' * H * z + h'*z)
        optimize!(model)
        z_Ipopt_L1_QP = [value(z[i]) for i in 1:n]
        y_Ipopt_L1_QP = -inv(Q)*(c+G'*z_Ipopt_L1_QP)
        Rel_err_Ipopt_L1_QP = [Rel_err_Ipopt_L1_QP; norm(y_Ipopt_L1_QP  - y)/norm(y)]
        Sol_time_Ipopt_L1_QP = [Sol_time_Ipopt_L1_QP; MOI.get(model, MOI.SolveTimeSec())]

        # Using ETCQP Solver to solve L1-QP
        Q_chol = cholesky(Q)
        L_Q_inv = Matrix(inv(Q_chol.L))
        V = G*L_Q_inv'
        epsilon = 1e-20
        y_ETC_L1_QP, sol_time_ETC_L1_QP = call_ETC_L1_QP(L_Q_inv, V, c, G, g, rho, epsilon, m, n)
        Rel_err_ETC_L1_QP = [Rel_err_ETC_L1_QP; norm(y_ETC_L1_QP-y)/norm(y)]
        Sol_time_ETC_L1_QP = [Sol_time_ETC_L1_QP; sol_time_ETC_L1_QP]
    end
    Worst_case_rel_err_Ipopt_L1_QP = [Worst_case_rel_err_Ipopt_L1_QP; maximum(Rel_err_Ipopt_L1_QP)]
    Worst_case_sol_time_Ipopt_L1_QP = [Worst_case_sol_time_Ipopt_L1_QP; maximum(Sol_time_Ipopt_L1_QP)]

    Worst_case_rel_err_ETC_L1_QP = [Worst_case_rel_err_ETC_L1_QP; maximum(Rel_err_ETC_L1_QP)]
    Worst_case_sol_time_ETC_L1_QP = [Worst_case_sol_time_ETC_L1_QP; maximum(Sol_time_ETC_L1_QP)]
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [3]:
println("Ipopt_L1_QP has relative error: ")
println(Worst_case_rel_err_Ipopt_L1_QP)
println("ETC_L1_QP has relative error: ")
println(Worst_case_rel_err_ETC_L1_QP)

println("Ipopt_L1_QP has copmutation time: ")
println(Worst_case_sol_time_Ipopt_L1_QP)
println("ETC_L1_QP has copmutation time: ")
println(Worst_case_sol_time_ETC_L1_QP)

Ipopt_L1_QP has relative error: 
Any[3.2886652266501874e-7, 9.112214886825521e-7, 7.301447534901695e-7, 1.4057458170413615e-6, 3.4482642613953084e-7, 1.752846832569202e-7, 1.2021909117898411e-6, 1.3947465766065672e-6, 5.975369548065534e-7, 4.133999636078562e-6, 2.261806997757935e-7, 7.006341608681003e-6, 2.6756134520014525e-5, 1.2197759376366483e-7, 4.886706349291374e-7, 8.519514623989731e-7, 1.0356494659384376e-6, 1.727985443145688e-5, 2.1164886422121525e-7, 1.9739817881622066e-7]
ETC_L1_QP has relative error: 
Any[1.913650394412832e-7, 2.607178536061794e-7, 4.442770894541579e-6, 4.6771257504139653e-7, 2.885282356077955e-7, 1.1976081931236238e-6, 4.390668524834006e-5, 6.438305724507412e-6, 7.996443730120402e-7, 3.0532827795160573e-6, 4.543044729589726e-7, 1.1563150958260575e-6, 2.3286636857685745e-5, 1.292387833165625e-7, 4.076535909850074e-6, 1.9215788669931992e-6, 5.237670055865379e-7, 1.1484898117307316e-5, 2.1157220628247515e-7, 4.967356522019114e-7]
Ipopt_L1_QP has copmutation ti

In [4]:
num_condition = 4
num_randomQPs = 4

Worst_case_rel_err_OSQP_L1_QP = []
Worst_case_sol_time_OSQP_L1_QP = []

for each_condition in 1:num_condition
    cond_num = 1e1 * (1e6/1e1)^((each_condition-1)/(num_condition-1))
    rho = 1 * cond_num .* ones(n);

    Rel_err_OSQP_L1_QP = []
    Sol_time_OSQP_L1_QP = []

    for each_QP in 1:num_randomQPs
        U = rand(m,m)
        Q = U*Diagonal(rand(1:cond_num,m))*U'
        Q = 0.5*(Q+Q')
        c = cond_num*vec(rand(m,1))
        G = sprandn(n,m,0.8)
        G = Matrix(G)
        g = rand(n)

        # the base solution is calculated by Ipopt in solving the Original-QP
        model_0 = Model(Ipopt.Optimizer)
        set_attribute(model_0, "print_level", 0)
        set_attribute(model_0, "tol", 1e-12)
        @variable(model_0, y[1:m])
        @objective(model_0, Min, 0.5 * y' * Q * y + c'*y)
        for i in 1:n
            @constraint(model_0, dot(G[i,:], y) <= g[i])
        end
        optimize!(model_0)
        y = [value(y[i]) for i in 1:m];
        
        # Using OSQP to solve the L1-QP
        H = G*inv(Q)*G'
        h = G*inv(Q)*c + g
        prob = OSQP.Model()
        OSQP.setup!(prob; P=sparse(H), q=h, A=sparse(Diagonal(ones(length(h)))), l=zeros(length(h)), u=rho, eps_abs=1e-20, eps_rel=1e-20, verbose=0, max_iter=1e5, polish=1)
        res = OSQP.solve!(prob)
        z_OSQP_L1_QP = res.x
        y_OSQP_L1_QP = -inv(Q)*(c+G'*z_OSQP_L1_QP)
        Rel_err_OSQP_L1_QP = [Rel_err_OSQP_L1_QP; norm(y_OSQP_L1_QP-y)/norm(y)]
        Sol_time_OSQP_L1_QP = [Sol_time_OSQP_L1_QP; res.info.run_time]
    end
    Worst_case_rel_err_OSQP_L1_QP = [Worst_case_rel_err_OSQP_L1_QP; maximum(Rel_err_OSQP_L1_QP)]
    Worst_case_sol_time_OSQP_L1_QP = [Worst_case_sol_time_OSQP_L1_QP; maximum(Sol_time_OSQP_L1_QP)]
end

In [5]:
println("OSQPL1_QP has relative error: ")
println(Worst_case_rel_err_OSQP_L1_QP)

println("OSQP_L1_QP has copmutation time: ")
println(Worst_case_sol_time_OSQP_L1_QP)


OSQPL1_QP has relative error: 
Any[1.7474949830440011e-7, 1.1939798972140675e-7, 1.0504796236689326e-7, 0.025028198667736767]
OSQP_L1_QP has copmutation time: 
Any[2.138161245, 2.109621077, 2.101454365, 2.1021723210000003]
